## Data prep

In [1]:
import pandas as pd
df = pd.read_csv("data/songdata.zip")
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
from pathlib import Path
import util

path = Path("chars.pkl")
chars = list()
if path.is_file():
    chars = util.load_vocab(path)
    print("Loaded from file")
else:
    vocab = set()
    for song in df["text"]:
        chars = set(song)
        vocab = vocab.union(chars)
    chars = list(vocab)
    util.write_vocab(path, chars)
    print("Generated from source")
    
vocab_size = len(chars)
print("Vocab size:", vocab_size)

Loaded from file
Vocab size: 76


In [9]:
char2idx = { char:i for i,char in enumerate(chars) }
idx2char = { i:char for i,char in enumerate(chars) }

In [10]:
def build_samples(song, buffer_length):
    tokens = song

    x_train = []
    y_train = []
    for i in range(0, len(song)):
        if i+buffer_length+1 >= len(tokens):
            continue
            
        x_train.append(tokens[i:i+buffer_length])
        y_train.append(tokens[i+buffer_length])

    return x_train,y_train

In [26]:
x_train, y_train = [], []
for song in df["text"][:500]:
    xs, ys = build_samples(song, 6)
    x_train.extend(xs)
    y_train.extend(ys)
print("Training data length:", len(x_train))
print("X[0]:", x_train[0])
print("Y[0]:", y_train[0])

Training data length: 581323
X[0]: Look a
Y[0]: t


## Train/Test split

In [27]:
from sklearn.model_selection import train_test_split
import math

# 80% Train, 10% Dev, 10% Test
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.2)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

In [28]:
import numpy as np
from random import shuffle

SEQUENCE_LENGTH = 6

def load_batch(xs, ys, begin, end):
    batch_size = end-begin
    
    x_train = np.zeros((batch_size, SEQUENCE_LENGTH, vocab_size))
    y_train = np.zeros((batch_size, vocab_size))
    
    xs_batch = xs[begin:end]
    ys_batch = ys[begin:end]
    
    c = list(zip(xs_batch, ys_batch))
    shuffle(c)
    xs_batch, ys_batch = zip(*c)
    
    for i in range(batch_size):
        x_train[i] = util.one_hot_encode_sequence(xs_batch[i], char2idx)
        y_train[i] = util.one_hot_encode(ys_batch[i], char2idx)
    
    return x_train, y_train

In [31]:
batches = util.generate_batches(len(X_train), 512)
begin, end = next(batches)
x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
print(x_batch.shape, y_batch.shape)

(512, 6, 76) (512, 76)


In [32]:
for i in range(10):
    xs = ''.join(util.one_hot_decode_sequence(x_batch[i], idx2char))
    y = util.one_hot_decode(y_batch[i], idx2char)
    print(f"{i} '{xs}' -> {y}")

0 'e's ju' -> s
1 'see yo' -> u
2 'e toni' -> g
3 ' show)' ->  
4 's  
In' ->  
5 'try to' ->  
6 't  
An' -> d
7 't you ' -> l
8 'ed you' ->  
9 'ap wit' -> h


## Training time!

In [34]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU
from keras.layers import LeakyReLU
from pathlib import Path

def build_model(vocab_size):
    model = Sequential()
    model.add(LSTM(512, input_shape=(SEQUENCE_LENGTH, vocab_size)))
    model.add(Dropout(0.5))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics = ['accuracy'])
    return model

model = build_model(vocab_size)
weights_path = Path("weights_char_rnn.h5")
if weights_path.is_file():
    print("Loaded!")
    model.load_weights(weights_path.resolve())
model.summary()

Loaded!
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 512)               1206272   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 76)                38988     
Total params: 1,245,260
Trainable params: 1,245,260
Non-trainable params: 0
_________________________________________________________________


In [45]:
batch_size = 8192
batches = generate_batches(len(X_train), batch_size)
print("Number of batches:", math.ceil(len(X_train) / batch_size))

Number of batches: 57


In [47]:
for begin, end in batches:
    x_batch, y_batch = load_batch(X_train, Y_train, begin, end)
    model.fit(x_batch, y_batch, batch_size=256, epochs=4)

Epoch 1/4
8192/8192 [==============================] - 1s 175us/step - loss: 1.7251 - accuracy: 0.5051
Epoch 2/4
8192/8192 [==============================] - 1s 180us/step - loss: 1.6197 - accuracy: 0.5283
Epoch 3/4
8192/8192 [==============================] - 1s 182us/step - loss: 1.5395 - accuracy: 0.5463
Epoch 4/4
8192/8192 [==============================] - 2s 184us/step - loss: 1.4483 - accuracy: 0.5706
Epoch 1/4
8192/8192 [==============================] - 2s 184us/step - loss: 1.7270 - accuracy: 0.5105
Epoch 2/4
8192/8192 [==============================] - 2s 187us/step - loss: 1.6027 - accuracy: 0.5378
Epoch 3/4
8192/8192 [==============================] - 1s 182us/step - loss: 1.4977 - accuracy: 0.5603
Epoch 4/4
8192/8192 [==============================] - 2s 187us/step - loss: 1.4241 - accuracy: 0.5819
Epoch 1/4
8192/8192 [==============================] - 2s 190us/step - loss: 1.7623 - accuracy: 0.5022
Epoch 2/4
8192/8192 [==============================] - 2s 198us/step - lo

8192/8192 [==============================] - 2s 185us/step - loss: 1.2835 - accuracy: 0.6218
Epoch 1/4
8192/8192 [==============================] - 1s 180us/step - loss: 1.6212 - accuracy: 0.5388
Epoch 2/4
8192/8192 [==============================] - 1s 183us/step - loss: 1.4662 - accuracy: 0.5741
Epoch 3/4
8192/8192 [==============================] - 2s 185us/step - loss: 1.3535 - accuracy: 0.5947
Epoch 4/4
8192/8192 [==============================] - 2s 188us/step - loss: 1.2686 - accuracy: 0.6206
Epoch 1/4
8192/8192 [==============================] - 1s 181us/step - loss: 1.5869 - accuracy: 0.5448
Epoch 2/4
8192/8192 [==============================] - 1s 183us/step - loss: 1.4381 - accuracy: 0.5792
Epoch 3/4
8192/8192 [==============================] - 2s 190us/step - loss: 1.3462 - accuracy: 0.6029
Epoch 4/4
8192/8192 [==============================] - 2s 191us/step - loss: 1.2450 - accuracy: 0.6257
Epoch 1/4
8192/8192 [==============================] - 1s 181us/step - loss: 1.6185

8192/8192 [==============================] - 2s 194us/step - loss: 1.2776 - accuracy: 0.6147
Epoch 4/4
8192/8192 [==============================] - 2s 184us/step - loss: 1.1931 - accuracy: 0.6387
Epoch 1/4
8192/8192 [==============================] - 1s 182us/step - loss: 1.5774 - accuracy: 0.5419
Epoch 2/4
8192/8192 [==============================] - 2s 184us/step - loss: 1.3949 - accuracy: 0.5930
Epoch 3/4
8192/8192 [==============================] - 2s 186us/step - loss: 1.2798 - accuracy: 0.6189
Epoch 4/4
8192/8192 [==============================] - 2s 197us/step - loss: 1.1998 - accuracy: 0.6421
Epoch 1/4
8192/8192 [==============================] - 2s 195us/step - loss: 1.5637 - accuracy: 0.5472
Epoch 2/4
8192/8192 [==============================] - 2s 188us/step - loss: 1.3963 - accuracy: 0.5885
Epoch 3/4
8192/8192 [==============================] - 2s 189us/step - loss: 1.2858 - accuracy: 0.6210
Epoch 4/4
8192/8192 [==============================] - 2s 199us/step - loss: 1.1948

In [61]:
model.save_weights("weights_char_rnn.h5")

In [70]:
from sampler import CharacterSampler
sampler = CharacterSampler(model, chars, SEQUENCE_LENGTH)

In [71]:
print(sampler.sample("i like", 700))

i like you half  
  
Where we ala, gaameloot  
And it  
'Cause I wouldn't know I need happy  
Walk on her in my that's the star nare  
Lasty farer and taI no on to  
The soul don't lang  
We on my stuld and wait  
I'll be cottone here  
Leavin' a little little walked and on a waane stod it all  
When you're fur  
Yes I'm this with you at amm  
But but mysnef it didn't toon's the best tound of tome on  
Nowne  
When you come to tell beding  
And when I could elent at I love some little bittle bit'm a little mishere, all on you beto eneed ot sleep alless will me  
  
Where we'll happy Hadmalatan words so tony ocess me ats me not, some time in the country knows that you reallt me betur beltor can lat


In [91]:
perplexity = 0.0
for x, y in list(zip(X_dev, Y_dev))[:800]:
    enc_seq = util.one_hot_encode_sequence(x, char2idx).reshape(1, SEQUENCE_LENGTH, vocab_size)
    preds = model.predict(enc_seq)
    y_hat = idx2char[np.argmax(preds)]
    likelihood = preds[0][np.argmax(y)]
    perplexity += np.log2(likelihood)
perplexity = np.power(2, perplexity * -1/800)
print(perplexity)

2019.3348552498442
